In [1]:
import polars as pl
import numpy as np
import json
import re
import numpy as np
from rapidfuzz import fuzz, process
from tqdm import tqdm

In [2]:
def extract_keywords(filepath):
    """
    Extracts unique keywords from a file with the format "x_keywords: [...]".
    """
    keywords = {}
    with open(filepath, 'r') as file:
        for line in file:
            match = re.match(r"(\w+_keywords)\s*=\s*\[(.*?)\]", line, re.DOTALL)
            if match:
                category = match.group(1)
                keyword_str = match.group(2)
                # Handle both single and double quotes, special characters,
                # remove extra whitespace, and filter out empty strings
                # The change is in the regex:
                keyword_list = [''.join(k).strip().strip("'").strip('"').strip("’") for k in re.findall(r"'([^']*)'|[\"]([^\"]*)[\"]|’([^’]*)’", keyword_str)]
                keyword_list = [keyword for keyword in keyword_list if keyword]
                keywords[category] = keyword_list

    # Flatten the dictionary and remove duplicates
    all_keywords = []
    for category, wordlist in keywords.items():
        all_keywords.extend(wordlist)

    return list(set(all_keywords))

root_folder = "/Users/antonina/Desktop/ADA/ada-2024-project-adacadabra2048" # change it for your root of the repo
stem_keywords = [keyword.lower() for keyword in 
                 extract_keywords(root_folder + '/src/scripts/keyword_generator/final_keywords.txt')]
len(stem_keywords)

2008

In [3]:
# Extract video data with country field (removing those without any country information)
df_path = '/Users/antonina/Desktop/ADA/edu-data/video_with_channelcountry.csv' # change it for your path
video_country = pl.read_csv(df_path).filter(pl.col("country").is_not_null())

In [4]:
def get_unique_tags(df: pl.DataFrame, tag_column: str = 'tags') -> list:
    return (df.lazy()
        .select(pl.col(tag_column))
        .filter(pl.col(tag_column).is_not_null())
        .select(
            pl.col(tag_column).str.split(',').alias('split_tags')
        )
        .explode('split_tags')
        .select(
            pl.col('split_tags').str.strip_chars()
        )
        .unique().collect()#.to_series()#.to_list()
    )

unique_tags = get_unique_tags(video_country)
type(unique_tags)

polars.dataframe.frame.DataFrame

In [8]:
match = process.extractOne('pho', stem_keywords, scorer=fuzz.ratio, score_cutoff=90)
none = (match == None)
if not none:
    print(match[1] > 0)
else:
    print(False)

False


In [9]:
def is_stem_tag(tag):
    """Checks if a tag is STEM"""
    tag = tag.lower().strip()
    # Using rapidfuzz's process.extractOne for faster fuzzy matching
    match = process.extractOne(tag, stem_keywords, scorer=fuzz.ratio, score_cutoff=90)
    none = (match == None)
    if not none:
        return match[1] > 0
    else:
        return False

In [10]:
unique_tags = unique_tags.with_columns(
    pl.col("split_tags").map_elements(lambda x: is_stem_tag(x)).alias("is_stem")
)

/var/folders/2y/m5zny5yd6990mwr5565hy46r0000gn/T/ipykernel_38878/674091713.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  unique_tags = unique_tags.with_columns(


In [12]:
len(unique_tags.filter(pl.col("is_stem") == True))

4989

In [14]:
len(unique_tags.filter(pl.col("is_stem") == False))

4124969

In [17]:
with open("stem_tags_fw90.json", 'w') as file:
    json.dump(unique_tags.filter(pl.col("is_stem") == True)['split_tags'].to_list(), file)